In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib qt

#the file for dispersal_1 is generated for Fig2 code which is in the Fig2 Folder. 
dispersal_1 = np.loadtxt("short_slim_0.25rep1sigma_dispersalrateestimates_seqlen_100_chopped_10000_rep_0.txt") 
#the file for dispersal_1_inf is generated using inf_cluster.py code in the same folder as this code.
dispersal_1_inf = np.loadtxt("slim_0.25rep1sigma_inf_dispersalrateestimates_seqlen_100_chopped_10000_rep_0.txt")

plt.plot(dispersal_1_inf[:,0],dispersal_1_inf[:,2], label = 'ARG Likelihood (d = $\infty$ )')

plt.plot(dispersal_1[:,0], dispersal_1[:,2], color='black', label = "ARG Likelihood")
#plt.plot(dispersal_1[:,0], dispersal_1[:,9], color = 'red', linestyle = "dashdot", label = "Average w/ Full ARG")
plt.plot(dispersal_1[:,0], dispersal_1[:,5], color = 'blue', linestyle = "--", label = "Comp Likelihood")
plt.plot(dispersal_1[:,0], dispersal_1[:,11], color = 'green', linestyle = "dotted", label = "Average (ARG)")
plt.plot(dispersal_1[21:len(dispersal_1[:,0]),0], dispersal_1[21:len(dispersal_1[:,0]),13], color = 'red', linestyle = "dotted", label = "W 20")

plt.axhline(y=0.25**2,label = "True Value", linestyle = '--', color = 'grey')
plt.axhline(y=0.25**2+0.5,label = "Effective Value", color = 'grey')

plt.legend()
plt.xlabel("Number of Trees", fontsize = 15)
plt.ylabel("Dispersal Estimate", fontsize = 15)
plt.show()



libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast
